## Create sample grid for training data (code in neural.py)

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [31]:
from Chempy.parameter import ModelParameters
from Chempy.cem_function import posterior_function_returning_predictions
from scipy.stats import norm as gaussian
import os
a = ModelParameters()

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
## This calculates a list of 5 trial values for each parameter around the prior value, as an array of 6 lists which will be combined
# Set the desired Gaussian sigma values in the widths parameter (values > prior sigma are used to fully explore parameter space)
# Parameter values are chosen that are evenly distributed in the Gaussian probability space (e.g. 16.7, 33, 50 etc. percentile points)

N = a.training_size # No. data points per parameter
widths = a.neural_widths # Gaussian widths for parameters

# Create 1d grid of data points equally spaced in probability space 
prob = np.linspace(1/(N+1), 1-1/(N+1), N)
grids = [gaussian.ppf(prob) for _ in range(N+1)] # Normalize to unit Gaussian
norm_grid = np.array(np.meshgrid(*grids)).T.reshape(-1,N+1)

# Create grid in parameter space
param_grid = [item*widths+a.p0 for item in norm_grid]

# Save grids
directory = 'Neural/'
if not os.path.exists(directory):
    os.makedirs(directory)
np.save(directory+'training_norm_grid.npy',norm_grid)
np.save(directory+'training_param_grid.npy')

In [52]:
# Create abundance output
#param_grid = param_grid[:6] # For testing
training_abundances = []
for i,item in enumerate(param_grid):
    abundances,_ = posterior_function_returning_predictions((item,a))
    training_abundances.append(abundances)
    if i%100 == 0:
        print("Calculating abundance set %d of %d" %(i,len(param_grid)))
              
# Save abundance table
np.save('Neural/training_abundances.npy', training_abundances)


Calculating abundance set 0 of 6


NOTES

- All code is now in neural.py file
- Should put changeable parameters e.g. number of choices for each parameter in parameter.py file - DONE
-  Find nicer way of using all rows from grid - DONE
- Check whether to use Karakas 10 or Karakas 16 - Karakas 10 for testing
- Automate number of traceable elements - just copy that from code - DONE

*This may be a useful reference https://arxiv.org/abs/1502.01852, for recommendation of ReLU units, with w = np.random.randn(n) x np.sqrt(2/n) for initialized weights, from Stanford course*

*Another useful reference: https://arxiv.org/abs/1412.6980 for 'Adam' learning method viz Stanford course*
